<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/SEIRS_comparacao1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from scipy.integrate import solve_ivp
import time

def seirs_disc(S0, E0, I0, R0, beta, sigma, gamma, omega, N, dt, T):
    passos = int(T/dt) + 1  # Número total de passos (inclui o tempo zero)
    S = np.zeros(passos)
    E = np.zeros(passos)
    I = np.zeros(passos)
    R = np.zeros(passos)
    t = np.linspace(0, T, passos)

    S[0] = S0
    E[0] = E0
    I[0] = I0
    R[0] = R0

    for k in range(passos - 1):
        dE = beta * (S[k] * I[k] / N) * dt
        dI = sigma * E[k] * dt
        dR = gamma * I[k] * dt
        dS = omega * R[k] * dt

        S[k+1] = S[k] - dE + dS
        E[k+1] = E[k] + dE - dI
        I[k+1] = I[k] + dI - dR
        R[k+1] = R[k] + dR - dS

    return I, t

def seirs_diff(t, y, beta, sigma, gamma, omega, N):
    S, E, I, R = y
    dS = -beta * (S * I / N) + omega * R
    dE = beta * (S * I / N) - sigma * E
    dI = sigma * E - gamma * I
    dR = gamma * I - omega * R
    return [dS, dE, dI, dR]

def solve_seirs(beta, sigma, gamma, omega, N, S0, E0, I0, R0, dt, T):
    y0 = [S0, E0, I0, R0]
    t_span = [0, T]
    t_eval = np.arange(0, T + dt, dt)
    t_eval = t_eval[t_eval <= T]

    sol = solve_ivp(seirs_diff, t_span, y0, args=(beta, sigma, gamma, omega, N), t_eval=t_eval)
    return sol.t, sol.y[2]

def plot_models(beta, sigma, gamma, omega, N, E0, I0, R0, dt_disc, dt_cont, T):
    S0 = N - E0 - I0 - R0

    # Mede o tempo de CPU para o modelo com equações de diferenças (discreto)
    start_disc = time.process_time()
    I_disc, t_disc = seirs_disc(S0, E0, I0, R0, beta, sigma, gamma, omega, N, dt_disc, T)
    cpu_disc = time.process_time() - start_disc

    # Mede o tempo de CPU para o modelo com equações diferenciais (contínuo)
    start_diff = time.process_time()
    t_diff, I_diff = solve_seirs(beta, sigma, gamma, omega, N, S0, E0, I0, R0, dt_cont, T)
    cpu_diff = time.process_time() - start_diff

    # Interpola os valores do modelo diferencial para os pontos do modelo discreto
    I_diff_interp = np.interp(t_disc, t_diff, I_diff)
    diff_perc = (np.abs(I_disc - I_diff_interp) / N) * 100

    # Plota os resultados
    fig, axs = plt.subplots(2, 1, figsize=(8, 10))

    axs[0].plot(t_disc, I_disc, label='I (Discreto)', color='red', linestyle='dashed')
    axs[0].plot(t_diff, I_diff, label='I (Diferencial)', color='blue')
    axs[0].set_title('Comparação dos Modelos SEIRS')
    axs[0].set_xlabel('Tempo (dias)')
    axs[0].set_ylabel('I(t)')
    axs[0].legend()
    axs[0].grid()

    axs[1].plot(t_disc, diff_perc, label='Diferença (%)', color='purple')
    axs[1].set_title('Diferença Relativa entre os Modelos')
    axs[1].set_xlabel('Tempo (dias)')
    axs[1].set_ylabel('ΔI/N (%)')
    axs[1].legend()
    axs[1].grid()

    plt.show()

    print("Tempo de CPU (Discreto): {:.5f} s".format(cpu_disc))
    print("Tempo de CPU (Diferencial): {:.5f} s".format(cpu_diff))

# Widgets Interativos
beta_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.3, description='Beta')
sigma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.2, description='Sigma')
gamma_slider = widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Gamma')
omega_slider = widgets.FloatSlider(min=0.001, max=0.1, step=0.001, value=0.01, description='Omega')
N_slider = widgets.IntSlider(min=100, max=10000, step=100, value=1000, description='N')
E0_slider = widgets.IntSlider(min=1, max=500, step=1, value=10, description='E0')
I0_slider = widgets.IntSlider(min=0, max=500, step=1, value=0, description='I0')
R0_slider = widgets.IntSlider(min=0, max=500, step=1, value=0, description='R0')
T_slider = widgets.IntSlider(min=50, max=1000, step=50, value=365, description='T')
dt_disc_slider = widgets.FloatSlider(min=0.05, max=1.0, step=0.05, value=0.5, description='dt Discreto')
dt_cont_slider = widgets.FloatSlider(min=0.02, max=2.0, step=0.02, value=0.1, description='dt Contínuo')

ui = widgets.VBox([
    beta_slider, sigma_slider, gamma_slider, omega_slider,
    N_slider, E0_slider, I0_slider, R0_slider,
    T_slider, dt_disc_slider, dt_cont_slider
])
out = widgets.interactive_output(plot_models, {
    'beta': beta_slider,
    'sigma': sigma_slider,
    'gamma': gamma_slider,
    'omega': omega_slider,
    'N': N_slider,
    'E0': E0_slider,
    'I0': I0_slider,
    'R0': R0_slider,
    'dt_disc': dt_disc_slider,
    'dt_cont': dt_cont_slider,
    'T': T_slider
})

display(ui, out)
